In [8]:
import time
import requests
import json
from datetime import datetime, timedelta

MAX_RETRIES = 3

start_date = datetime(2024, 7, 18)
end_date = datetime(2024, 7, 19)

# Define the list of events
events = ["ApplySuccess"]  # Add your events here

def get_event_data(event_name, date_integer):
    url = "https://in1.api.clevertap.com/1/profiles.json?batch_size=20000"
    payload = json.dumps({
      "event_name": event_name,
      "from": date_integer,
      "to": date_integer
    })
    headers = {
      'X-CleverTap-Account-Id': '*********',
      'X-CleverTap-Passcode': '***********',
      'Content-Type': 'application/json'
     }

    total_users = 0
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            response = requests.post(url, headers=headers, data=payload)
            response.raise_for_status()  # Raise an HTTPError for bad responses

            response_json = response.json()
            cursor = response_json.get('cursor')

            while cursor:
                # Count users only for the specified event
                for record in response_json.get('records', []):
                     if 'events' in record and event_name in record['events']:
                            total_users += 1

                # If there's a cursor, fetch the next set of results
                next_url = f"https://in1.api.clevertap.com/1/profiles.json?cursor={cursor}"
                response = requests.get(next_url, headers=headers)
                response.raise_for_status()

                response_json = response.json()
                cursor = response_json.get('next_cursor')

            return total_users

        except requests.RequestException as e:
            print(f"Request failed: {e}")

        retry_count += 1
        print(f"Retrying... Attempt {retry_count}")
        time.sleep(5 * retry_count)

    print("Max retries reached. Unable to fetch data.")
    return 0

current_date = start_date
all_results = []

while current_date <= end_date:
    date_integer = int(current_date.strftime('%Y%m%d'))

    for event in events:
        total_users = get_event_data(event, date_integer)
        all_results.append({
         'Date': current_date.strftime('%Y-%m-%d'),
         'Event': event,
         'TotalUsers': total_users
         })

    current_date += timedelta(days=1)
    
# Convert all_results to a pandas DataFrame
df = pd.DataFrame(all_results)
df

,Date,Event,TotalUsers
0,2024-07-18,ApplySuccess,20184
1,2024-07-19,ApplySuccess,18979
